In [ ]:
# !pip install datasets jiwer g2p g2p_en
# Import libraries
# !pip3 install torch torchvision torchaudio
# !pip show transformers
# !pip install accelerate -U
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from jiwer import wer,cer
from huggingface_hub import notebook_login
import accelerate
import os
import pandas as pd
from g2p_en import G2p
import re
from huggingface_hub import Repository

print(accelerate.__version__)

In [1]:
col_names = ['predictions', 'references']
phoneme_col_names = ['predictions_phoneme', 'references_phoneme']
SPEAKERS = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']
# Define the source and target language columns
source_lang = 'predictions_phoneme'
target_lang = 'references_phoneme'
max_length = 512

from enum import Enum

class TorgoTrainType(Enum):
    WORD_NO_KEEP = "word_no_keep"
    WORD_KEEP = "word_keep"

# BART

In [ ]:
def print_dataframe_sizes(dataframes_dict):
    for df_name, df in dataframes_dict.items():
        print(f"{df_name} - Data Size: {len(df)}")
    print()

In [ ]:
# Tokenize the data
# The preprocess_function function is defined to preprocess the data by tokenizing the inputs and labels
def preprocess_function(examples):
    inputs = [f'{source_lang}: {text}' for text in examples[source_lang]]
    targets = examples[target_lang]
    encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
    model_inputs = {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': tokenizer(targets, padding=True, truncation=True, return_tensors='pt')['input_ids'].squeeze()
    }
    return model_inputs

# define a data_collator function for batch processing
def data_collator(features):
    batch = {}
    # Pad input_ids and attention_mask to the maximum length within the batch
    max_length = max(len(feature['input_ids']) for feature in features)
    batch['input_ids'] = torch.stack([torch.tensor(feature['input_ids'] + [tokenizer.pad_token_id] * (max_length - len(feature['input_ids']))) for feature in features])
    batch['attention_mask'] = torch.stack([torch.tensor(feature['attention_mask'] + [0] * (max_length - len(feature['attention_mask']))) for feature in features])
    batch['labels'] = torch.stack([torch.tensor(feature['labels'] + [-100] * (max_length - len(feature['labels']))) for feature in features])
    return batch